<a href="https://colab.research.google.com/github/vikashsrivastava94/google_colab_2/blob/master/nifty_direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re

In [0]:
df=pd.read_csv("C:/Python projects/nifty project/fwddata/dat/NIFTY.csv")

In [0]:
df.head()

,Date,Open,High,Low,Close,Volume,Turnover
0,2010-01-04,5200.90,5238.45,5167.10,5232.20,148652424,6.531610e+10
1,2010-01-05,5277.15,5288.35,5242.40,5277.90,240844424,7.969620e+10
2,2010-01-06,5278.15,5310.85,5260.05,5281.80,216147837,7.892600e+10
3,2010-01-07,5281.80,5302.55,5244.75,5263.10,181246734,6.890990e+10
4,2010-01-08,5264.25,5276.75,5234.70,5244.75,201910800,7.777040e+10


In [0]:
df.shape

(2320, 7)

In [0]:
# Create the dependent variable
df.loc[(df['Close']-df['Close'].shift(-1)) < 0,'target']=1
df.loc[(df['Close']-df['Close'].shift(-1)) > 0,'target']=0

In [0]:
df.tail()

,Date,Open,High,Low,Close,Volume,Turnover,target,CCI_5,CCI_10,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
2314,2019-05-02,11725.55,11789.30,11699.55,11724.75,380278045,1.779006e+11,0.0,73.178583,40.141471,...,2,3,122,0,0,0,0,0,0,1556755200
2315,2019-05-03,11722.60,11770.90,11699.35,11712.25,305519934,1.515632e+11,0.0,26.458732,23.133281,...,3,4,123,0,0,0,0,0,0,1556841600
2316,2019-05-06,11605.80,11632.55,11571.35,11598.25,299046480,1.470342e+11,0.0,-118.516187,-95.709232,...,6,0,126,0,0,0,0,0,0,1557100800
2317,2019-05-07,11651.50,11657.05,11484.45,11497.90,337495624,1.663284e+11,0.0,-91.780609,-118.423007,...,7,1,127,0,0,0,0,0,0,1557187200
2318,2019-05-08,11478.70,11479.10,11346.95,11359.45,372826025,1.744001e+11,1.0,-97.241575,-150.655635,...,8,2,128,0,0,0,0,0,0,1557273600


In [0]:
df.target.value_counts()

0.0    1221
1.0    1095
Name: target, dtype: int64

In [0]:
df.drop(df.tail(1).index,inplace=True)

In [0]:
# Creating technical indicators

# 1. CCI (Commodity Channel Index)
def CCI(data, ndays): 
    TP = (data['High'] + data['Low'] + data['Close']) / 3 
    CCI = pd.Series((TP - TP.rolling(ndays).mean()) / (0.015 * TP.rolling(ndays).std()),name = 'CCI') 
    data['CCI_'+str(ndays)] = pd.Series(CCI, index=data.index) 
    return data

# Compute the Commodity Channel Index(CCI) for NIFTY based on the 20-day Moving average
for day in [5,10,20,60]:
    df=CCI(df,day)

In [0]:
df.head(20)

,Date,Open,High,Low,Close,Volume,Turnover,target,CCI_5,CCI_10,CCI_20,CCI_60
0,2010-01-04,5200.90,5238.45,5167.10,5232.20,148652424,6.531610e+10,0.0,NaN,NaN,NaN,NaN
1,2010-01-05,5277.15,5288.35,5242.40,5277.90,240844424,7.969620e+10,0.0,NaN,NaN,NaN,NaN
2,2010-01-06,5278.15,5310.85,5260.05,5281.80,216147837,7.892600e+10,1.0,NaN,NaN,NaN,NaN
3,2010-01-07,5281.80,5302.55,5244.75,5263.10,181246734,6.890990e+10,1.0,NaN,NaN,NaN,NaN
4,2010-01-08,5264.25,5276.75,5234.70,5244.75,201910800,7.777040e+10,0.0,-13.596443,NaN,NaN,NaN
5,2010-01-11,5263.80,5287.20,5227.80,5249.40,238011959,1.108055e+11,1.0,-57.999728,NaN,NaN,NaN
6,2010-01-12,5251.10,5300.50,5200.95,5210.40,206748015,8.648490e+10,0.0,-83.052305,NaN,NaN,NaN
7,2010-01-13,5212.60,5239.20,5169.55,5233.95,200774550,8.430480e+10,0.0,-99.417783,NaN,NaN,NaN
8,2010-01-14,5234.50,5272.85,5232.50,5259.90,171282618,7.824430e+10,1.0,47.158009,NaN,NaN,NaN
9,2010-01-15,5259.90,5279.85,5242.45,5252.20,167242355,6.927870e+10,0.0,51.320173,20.980257,NaN,NaN


In [0]:
# 2. Ease of movement EMV

def EMV(data, ndays): 
    dm = ((data['High'] + data['Low'])/2) - ((data['High'].shift(1) + data['Low'].shift(1))/2)
    br = (data['Volume'] / 100000000) / ((data['High'] - data['Low']))
    EMV = dm / br 
    EMV_MA=pd.Series(EMV.rolling(ndays).mean())
    data['EMV_MA_'+str(ndays)] = pd.Series(EMV_MA, index=data.index)  
    return data

for day in [5,10,20,60,150,240]:
    df=EMV(df,day)

In [0]:
df.head(50)

In [0]:
#@title Default title text
# Splitting data into meaningful features to capture the idea of time series behavior

def add_datepart(df, fldname):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    return df

df=add_datepart(df,'Date')

# Convert boolean cols to int type
bool_cols=[col for col in df.columns if df[col].dtypes.name == 'bool']
for col in bool_cols:
    df[col] = df[col].astype(int)

In [0]:
df.head()

,Date,Open,High,Low,Close,Volume,Turnover,target,CCI_5,CCI_10,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2010-01-04,5200.90,5238.45,5167.10,5232.20,148652424,6.531610e+10,0.0,NaN,NaN,...,4,0,4,0,0,0,0,0,0,1262563200
1,2010-01-05,5277.15,5288.35,5242.40,5277.90,240844424,7.969620e+10,0.0,NaN,NaN,...,5,1,5,0,0,0,0,0,0,1262649600
2,2010-01-06,5278.15,5310.85,5260.05,5281.80,216147837,7.892600e+10,1.0,NaN,NaN,...,6,2,6,0,0,0,0,0,0,1262736000
3,2010-01-07,5281.80,5302.55,5244.75,5263.10,181246734,6.890990e+10,1.0,NaN,NaN,...,7,3,7,0,0,0,0,0,0,1262822400
4,2010-01-08,5264.25,5276.75,5234.70,5244.75,201910800,7.777040e+10,0.0,-13.596443,NaN,...,8,4,8,0,0,0,0,0,0,1262908800


In [0]:
df=df=add_datepart(df,'Date')

In [0]:
# Replacing the raw prices with their daily returns

df['Open']=df['Open']-df['Open'].shift(1)
df['High']=df['High']-df['High'].shift(1)
df['Low']=df['Low']-df['Low'].shift(1)
df['Close']=df['Close']-df['Close'].shift(1)

In [0]:
df.head()

,Date,Open,High,Low,Close,Volume,Turnover,target,CCI_5,CCI_10,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2010-01-04,NaN,NaN,NaN,NaN,148652424,6.531610e+10,1.0,NaN,NaN,...,4,0,4,0,0,0,0,0,0,1262563200
1,2010-01-05,76.25,49.9,75.30,45.70,240844424,7.969620e+10,1.0,NaN,NaN,...,5,1,5,0,0,0,0,0,0,1262649600
2,2010-01-06,1.00,22.5,17.65,3.90,216147837,7.892600e+10,0.0,NaN,NaN,...,6,2,6,0,0,0,0,0,0,1262736000
3,2010-01-07,3.65,-8.3,-15.30,-18.70,181246734,6.890990e+10,0.0,NaN,NaN,...,7,3,7,0,0,0,0,0,0,1262822400
4,2010-01-08,-17.55,-25.8,-10.05,-18.35,201910800,7.777040e+10,1.0,-13.596443,NaN,...,8,4,8,0,0,0,0,0,0,1262908800


### Splitting data into train and test

In [0]:
def temporal_splitting(data,date1,date2):
    train=data.loc[data['Date'] < date1,:]
    val=data.loc[(data['Date'] >= date1) & (data['Date'] < date2),:]
    test=data.loc[data['Date'] >= date2]
    
    return train, val, test

In [0]:
train, val,test=temporal_splitting(df,'2017-01-01','2018-05-08')

In [0]:
train.shape,val.shape,test.shape

((1739, 31), (333, 31), (247, 31))

In [0]:
def my_var_columns(data,cols_to_remove):
    vars__ = list(data.columns[~data.columns.isin(cols_to_remove)])
    return vars__

In [0]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover', 'target',
       'CCI_5', 'CCI_10', 'CCI_20', 'CCI_60', 'EMV_MA_5', 'EMV_MA_10',
       'EMV_MA_20', 'EMV_MA_60', 'EMV_MA_150', 'EMV_MA_240', 'Year', 'Month',
       'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'Elapsed'],
      dtype='object')

In [0]:
DV='target'

# Drop columns which are not required
cols_to_remove=['Date',DV,'Turnover']

In [0]:
train_cols = my_var_columns(df,cols_to_remove)

In [0]:
train_cols

['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'CCI_5',
 'CCI_10',
 'CCI_20',
 'CCI_60',
 'EMV_MA_5',
 'EMV_MA_10',
 'EMV_MA_20',
 'EMV_MA_60',
 'EMV_MA_150',
 'EMV_MA_240',
 'Year',
 'Month',
 'Week',
 'Day',
 'Dayofweek',
 'Dayofyear',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start',
 'Elapsed']

In [0]:
X_train=train[train_cols]
X_val=val[train_cols]
X_test=test[train_cols]

Y_train=train[DV]
Y_val=val[DV]
Y_test=test[DV]

In [0]:
X_train.tail()

,Open,High,Low,Close,Volume,CCI_5,CCI_10,CCI_20,CCI_60,EMV_MA_5,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
1734,7965.10,7970.05,7893.80,7908.25,110171222,-87.291530,-116.236720,-161.196720,-106.900721,-2273.544635,...,26,0,361,0,0,0,0,0,0,1482710400
1735,7915.05,8044.65,7903.70,8032.85,122463312,-1.258050,-55.292972,-94.845697,-86.646411,-1112.791185,...,27,1,362,0,0,0,0,0,0,1482796800
1736,8047.55,8100.55,8028.40,8034.85,130580701,91.970731,0.138721,-44.937387,-69.373257,-17.049045,...,28,2,363,0,0,0,0,0,0,1482883200
1737,8030.60,8111.10,8020.80,8103.60,183984077,78.007352,31.929433,-22.931032,-61.494673,992.892181,...,29,3,364,0,0,0,0,0,0,1482969600
1738,8119.65,8197.00,8114.75,8185.80,131457250,89.826316,110.094271,41.165853,-38.254535,2412.551668,...,30,4,365,0,0,0,0,0,0,1483056000


In [0]:
print(Y_train.value_counts())
print(Y_val.value_counts())
print(Y_test.value_counts())

0.0    904
1.0    833
Name: target, dtype: int64
0.0    187
1.0    145
Name: target, dtype: int64
0.0    130
1.0    117
Name: target, dtype: int64


In [0]:
# Impute the missing values
X_train=X_train.fillna(X_train.mean())
X_val=X_val.fillna(X_train.mean())
X_test=X_test.fillna(X_train.mean())

In [0]:
X_train.dtypes

Open                float64
High                float64
Low                 float64
Close               float64
Volume                int64
CCI_5               float64
CCI_10              float64
CCI_20              float64
CCI_60              float64
EMV_MA_5            float64
EMV_MA_10           float64
EMV_MA_20           float64
EMV_MA_60           float64
EMV_MA_150          float64
EMV_MA_240          float64
Year                  int64
Month                 int64
Week                  int64
Day                   int64
Dayofweek             int64
Dayofyear             int64
Is_month_end          int32
Is_month_start        int32
Is_quarter_end        int32
Is_quarter_start      int32
Is_year_end           int32
Is_year_start         int32
Elapsed               int64
dtype: object

In [0]:
df.dropna(subset=[DV], how='all', inplace = True)

### Training XgBoost classifier

In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit

import xgboost as xgb
import lightgbm as lgb
import bayes_opt as bayso
from sklearn.metrics import accuracy_score,confusion_matrix

In [0]:
def lgbcv1(min_child_weight,
           colsample_bytree,
           max_depth,
           subsample,
           min_gain_to_split,
           reg_alpha,
           reg_lambda):

    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting': 'gbdt',
        'random_state' : 501, # Updated from 'seed'
        'n_estimators' : 150,
        'learning_rate': 0.01,
        'early_stopping_round' : 10,
        'verbose': 20,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'bagging_freq': 1,
        'bagging_seed': RS,
        'feature_fraction_seed': RS,
        'max_bin': 50,
        'min_child_weight': int(min_child_weight),
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'max_depth': int(max_depth),
        'subsample': max(min(subsample, 1), 0),
        'min_gain_to_split': max(min_gain_to_split,0),
        'reg_alpha': max(reg_alpha,0),
        'reg_lambda': max(reg_lambda,0)
    }


    cv_result = lgb.cv(params, lgtrain, nfold=5, seed=0, verbose_eval =200,stratified=False,shuffle=False)

    return max(cv_result['auc-mean'])

In [0]:
def prepare_data():
    lgtrain = lgb.Dataset(X_train, label=Y_train)

    return lgtrain

In [0]:
if __name__ == '__main__':
    lgtrain =  prepare_data()
    random_state = 2019
    init_points = 60
    numitems=30
    RS = 20190523
    np.random.seed(RS)
    lgbo = bayso.BayesianOptimization(lgbcv1,
                                      {'min_child_weight': (1, 20),
                                      'colsample_bytree': (0.8, 1),
                                      'max_depth': (3, 5),
                                      'subsample': (0.3, 1.0),
                                      'min_gain_to_split': (0, 20),
                                      'reg_alpha': (0, 10),
                                      'reg_lambda': (0, 100)})
    
    lgbo.maximize(init_points=init_points, n_iter=numitems)

|   iter    |  target   | colsam... | max_depth | min_ch... | min_ga... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.4989   |  0.8674   |  3.389    |  15.66    |  8.634    |  3.469    |  76.81    |  0.4983   |
|  2        |  0.4989   |  0.8561   |  3.529    |  6.769    |  7.025    |  7.794    |  29.85    |  0.4963   |
|  3        |  0.4989   |  0.9416   |  4.91     |  2.967    |  17.92    |  4.406    |  34.09    |  0.782    |
|  4        |  0.4989   |  0.923    |  4.298    |  14.72    |  7.07     |  2.418    |  35.23    |  0.5627   |
|  5        |  0.4989   |  0.9248   |  4.394    |  13.18    |  10.03    |  8.152    |  43.12    |  0.313    |
|  6        |  0.5117   |  0.9146   |  4.68     |  6.7      |  2.276    |  2.084    |  37.63    |  0.3435   |
|  7        |  0.4989   |  0.9076   |  3.999    |  13.78    |  12.26    |  9.835    |  54.83    |  0.3512   |
|  8      

|  45       |  0.4989   |  0.9078   |  4.538    |  1.141    |  14.9     |  8.614    |  1.939    |  0.7017   |
|  46       |  0.4989   |  0.8193   |  4.513    |  3.145    |  11.89    |  2.751    |  18.87    |  0.8959   |
|  47       |  0.4989   |  0.8238   |  4.913    |  9.321    |  15.16    |  1.045    |  77.93    |  0.3591   |
|  48       |  0.4989   |  0.9015   |  3.22     |  12.78    |  4.512    |  6.284    |  74.34    |  0.4701   |
|  49       |  0.4989   |  0.8384   |  4.772    |  14.42    |  10.52    |  7.875    |  35.46    |  0.3289   |
|  50       |  0.4989   |  0.8028   |  3.786    |  3.08     |  10.11    |  2.881    |  27.97    |  0.6433   |
|  51       |  0.4846   |  0.9659   |  4.549    |  16.21    |  4.046    |  5.551    |  2.566    |  0.4323   |
|  52       |  0.4989   |  0.8823   |  3.561    |  1.72     |  9.06     |  6.331    |  51.94    |  0.5138   |
|  53       |  0.4989   |  0.8937   |  3.511    |  10.33    |  9.131    |  4.387    |  52.42    |  0.9159   |
|  54     

|  90       |  0.5314   |  0.9653   |  3.101    |  19.22    |  0.1452   |  0.02356  |  77.87    |  0.9053   |


In [0]:
lgbo.res

[{'target': 0.49891419491525424,
  'params': {'colsample_bytree': 0.8674486612030401,
   'max_depth': 3.389357418736848,
   'min_child_weight': 15.664398724508612,
   'min_gain_to_split': 8.633856483986035,
   'reg_alpha': 3.4691987334898258,
   'reg_lambda': 76.81304453217209,
   'subsample': 0.4983353175713623}},
 {'target': 0.49891419491525424,
  'params': {'colsample_bytree': 0.8560993979534014,
   'max_depth': 3.528708299488007,
   'min_child_weight': 6.768782399516305,
   'min_gain_to_split': 7.025060859900531,
   'reg_alpha': 7.7941124213820245,
   'reg_lambda': 29.8481495639883,
   'subsample': 0.4962821950864367}},
 {'target': 0.49891419491525424,
  'params': {'colsample_bytree': 0.9415757534264778,
   'max_depth': 4.910240661952569,
   'min_child_weight': 2.966958433218232,
   'min_gain_to_split': 17.917717690393083,
   'reg_alpha': 4.406211897778432,
   'reg_lambda': 34.091920966488445,
   'subsample': 0.7820140680738819}},
 {'target': 0.49891419491525424,
  'params': {'cols

In [0]:
max_target=max([lgbo.res[elem]['target'] for elem in range(len(lgbo.res))])

In [0]:
opt_params=[elem['params'] for elem in lgbo.res if elem['target']==max_target][0]

In [0]:
opt_params.update({
    'objective': 'binary',
    'metric': 'auc',
    'boosting': 'gbdt',
    'random_state' : 501, # Updated from 'seed'
    'n_estimators' : 150,
    'learning_rate': 0.01,
    'early_stopping_round' : 10,
    'verbose': 20,
    'num_leaves': 31,
    'min_data_in_leaf': 50,
    'bagging_freq': 1,
    'bagging_seed': RS,
    'feature_fraction_seed': RS,
    'max_bin': 50})
opt_params['min_child_weight'] = int(opt_params['min_child_weight'])
opt_params['colsample_bytree'] = max(min(opt_params['colsample_bytree'], 1), 0)
opt_params['max_depth'] = int(opt_params['max_depth'])
opt_params['subsample'] = max(min(opt_params['subsample'], 1), 0)
opt_params['min_gain_to_split'] = max(opt_params['min_gain_to_split'],0)
opt_params['reg_alpha'] = max(opt_params['reg_alpha'],0)
opt_params['reg_lambda'] = max(opt_params['reg_lambda'],0)

In [0]:
opt_params

{'colsample_bytree': 0.9653331046182841,
 'max_depth': 3,
 'min_child_weight': 19,
 'min_gain_to_split': 0.14518323687155776,
 'reg_alpha': 0.023560049967107366,
 'reg_lambda': 77.86659816312816,
 'subsample': 0.9053295884953352,
 'objective': 'binary',
 'metric': 'auc',
 'boosting': 'gbdt',
 'random_state': 501,
 'n_estimators': 150,
 'learning_rate': 0.01,
 'early_stopping_round': 10,
 'verbose': 20,
 'num_leaves': 31,
 'min_data_in_leaf': 50,
 'bagging_freq': 1,
 'bagging_seed': 20190523,
 'feature_fraction_seed': 20190523,
 'max_bin': 50}

In [0]:
lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval = lgb.Dataset(X_val, Y_val, reference=lgb_train)

In [0]:
import lightgbm as lgb

In [0]:
lgb_model = lgb.train(opt_params, lgb_train, valid_sets=lgb_eval)

[1]	valid_0's auc: 0.507098
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.541196
[3]	valid_0's auc: 0.486959
[4]	valid_0's auc: 0.463481
[5]	valid_0's auc: 0.477751
[6]	valid_0's auc: 0.473716
[7]	valid_0's auc: 0.483841
[8]	valid_0's auc: 0.47445
[9]	valid_0's auc: 0.476339
[10]	valid_0's auc: 0.476853
[11]	valid_0's auc: 0.478668
[12]	valid_0's auc: 0.47201
Early stopping, best iteration is:
[2]	valid_0's auc: 0.541196


In [0]:
train_preds_lgb=lgb_model.predict(X_train)
val_preds_lgb=lgb_model.predict(X_val)
test_preds_lgb=lgb_model.predict(X_test)

In [0]:
train_preds_lgb

array([0.47994717, 0.47994717, 0.47994717, ..., 0.47824794, 0.47875097,
       0.47875097])

In [0]:
train_preds_lgb=[np.argmax(line) for line in range(len(train_preds_lgb))]


In [0]:
test_preds_lgb=[np.argmax(line) for line in range(len(test_preds_lgb))]

In [0]:
type(train_preds_lgb)

list

In [0]:
print(roc_auc_score(Y_train,train_preds_lgb))
print(roc_auc_score(Y_test,test_preds_lgb))

ValueError: continuous format is not supported

In [0]:
from sklearn.metrics import auc